In [1]:
import numpy as np
import mir_eval.transcription
import os
from IPython.display import display, Markdown
from midiutil import MIDIFile
from scipy.signal import find_peaks

from transcribe_factorization import transcribe_activations_dynamic_AD
import transcribe_factorization as tf
import evaluate_transcription as et
import STFT
import pandas as pd

ModuleNotFoundError: No module named 'mir_eval'

In [18]:
def printmd(string):
    display(Markdown(string))

def compute_scores_database(piano_type, H_normalization = False, fusion_peak = False):
    path_computed_H = "../data_persisted/activation_AD"
    path_songs = "/Volumes/TOSHIBA/MAPS/{}/MUS".format(piano_type)
    
    time_limit = 30
    beta = 1
    model_AD = True
    note_intensity = "M"

    codebook = range(21, 109)
    onset_tolerance = 50/1000
    
#     a = np.arange(1e-3, 1e-2, 1e-3)
#     b = np.arange(1e-4, 1e-3, 1e-4)
#     c = np.arange(1e-5, 1e-4, 1e-5)
#     f = np.arange(3e-1, 8e-1, 2e-2)
    f = np.arange(1e-3, 1e-1, 1e-3)
#     f = np.arange(1e-2, 6e-1, 2e-2)

    listthres = np.r_[f]
    files = os.listdir(path_songs)
    list_files_wav = []
    for it_files in files:
        if it_files.split(".")[-1] == "wav" and it_files[0] == "M":
            list_files_wav.append(it_files)

    all_res = []
    for a_song in list_files_wav:
#         song_name = a_song.replace(".wav", "")
        song_name = a_song[:-4]
#         print("processing piano song: {}".format(song_name))
#         path_this_song = "{}/{}".format(path_songs, a_song)
        path_this_song = path_songs + "/" + a_song
        stft = STFT.STFT(path_this_song, time = time_limit, model_AD=model_AD, channel = 0)

        X = stft.get_magnitude_spectrogram()

#         annot_name = a_song.replace("wav","txt")
#         annot_this_song = "{}/{}".format(path_songs, annot_name)
        annot_this_song = path_songs + "/" + a_song[:-3] + "txt"
        note_annotations = et.load_ref_in_array(annot_this_song, time_limit=time_limit)
        ref = np.array(note_annotations, float)
        ref_pitches = np.array(ref[:,2], int)
        try:
            #res_each_song = []
            res_a_param = []
            for mode in ["attack", "normal"]:
                H_persisted_name = "AD_{}_activations_song_{}_W_learned_{}_beta_{}_stftAD_{}_intensity_W_{}_time_limit_{}".format(mode, song_name, piano_type, beta, model_AD, note_intensity, time_limit)
                H = np.load("{}/{}.npy".format(path_computed_H, H_persisted_name), allow_pickle = True)
                res_every_thresh = []
                for threshold in listthres:
                    if fusion_peak:
                        prediction, _ = transcribe_activations_dynamic_AD(codebook, H, stft, threshold, H_normalization = H_normalization)
                    else:
                        prediction, _ = tf.transcribe_activations_dynamic(codebook, H, stft, threshold, H_normalization = H_normalization)
                    est = np.array(prediction, float)
                    if est.size > 0:
                        est_pitches = np.array(est[:,2], int)
                        (prec, rec, f_mes, _) = mir_eval.transcription.precision_recall_f1_overlap(ref[:,0:2], ref_pitches, est[:,0:2], est_pitches, offset_ratio = None, onset_tolerance = onset_tolerance)
                        matching = mir_eval.transcription.match_notes(ref[:,0:2], ref_pitches, est[:,0:2],est_pitches, onset_tolerance=onset_tolerance,offset_ratio=None)
                        TP = len(matching)
                        try:
                            FP = int(TP * (1 - prec) / prec)
                        except ZeroDivisionError:
                            FP = 0
                        try:
                            FN = int(TP * (1 - rec) / rec)
                        except ZeroDivisionError:
                            FN = 0
                        acc = et.accuracy(TP,FP,FN)
                    else:
                        prec, rec, f_mes, acc, TP, FP, FN = (0,0,0,0,0,0,0)
                    res_every_thresh.append([prec, rec, f_mes, acc, TP, FP, FN])
                res_a_param.append(res_every_thresh)
                res_each_song = np.array(res_every_thresh)
                best_ind = np.argmax(res_each_song[:, 2])
            all_res.append(res_a_param)

        except FileNotFoundError:
            print("\033[91m This song failed: {} \033[00m".format(a_song)) 
            pass
    np_all_res = np.array(all_res)
    the_t = []
    for t in ["attack", "normal"]:
        the_t.append("H: {}".format(t))
    index_pandas = the_t
    col = ['Best threshold','Precision', 'Recall', 'F measure','Accuracy','True Positives','False Positives','False Negatives']
    lines = []
    lines_opt_thresh = []
    for cond in range(len(index_pandas)):
        all_thresh = []
        for each_thresh in range(len(listthres)):
            all_thresh.append(np.mean(np_all_res[:,cond,each_thresh,2]))
        best_thresh_idx = np.argmax(all_thresh)
        this_line = [listthres[best_thresh_idx]]
        for i in range(len(col) - 1):# - 1 because threshold
            this_line.append(round(np.mean(np_all_res[:,cond,best_thresh_idx,i]), 4))
        lines.append(this_line)
        
        best_val = []
        for each_song in range(len(list_files_wav)):
            best_thresh_idx = np.argmax(np_all_res[each_song,cond,:,2])
            best_val.append([round(np_all_res[each_song,cond,best_thresh_idx,i], 4) for i in range(len(col) - 1)])
        lines_opt_thresh.append([round(np.mean(np.array(best_val)[:,i]),4) for i in range(len(col) - 1)])
    
    print("Piano: {}".format(piano_type))
    print("When averaging each threshold on all MAPS")
    df = pd.DataFrame(np.array(lines), columns = col, index = index_pandas)
    display(df.style.bar(subset=["F measure", "Accuracy"], color='#5fba7d'))

    print("When optimizing the threshold on each song")
    best_df = pd.DataFrame(np.array(lines_opt_thresh), columns = col[1:], index = index_pandas)
    display(best_df.style.bar(subset=["F measure", "Accuracy"], color='#5fba7d'))

In [19]:
for piano_type in ["AkPnCGdD","ENSTDkCl","AkPnBcht","AkPnBsdf","AkPnStgb","ENSTDkAm","SptkBGAm","StbgTGd2"]:
    compute_scores_database(piano_type, H_normalization = False, fusion_peak= False)
    

Piano: AkPnCGdD
When averaging each threshold on all MAPS


,Best threshold,Precision,Recall,F measure,Accuracy,True Positives,False Positives,False Negatives
H: attack,0.072000,0.538200,0.689700,0.597000,0.428600,157.866700,138.033300,93.566700
H: normal,0.031000,0.715600,0.789300,0.743900,0.599700,185.666700,76.233300,65.633300


When optimizing the threshold on each song


,Precision,Recall,F measure,Accuracy,True Positives,False Positives,False Negatives
H: attack,0.540400,0.729100,0.616600,0.449100,169.900000,149.300000,81.600000
H: normal,0.757800,0.790000,0.771000,0.634700,189.100000,66.300000,62.100000


Piano: ENSTDkCl
When averaging each threshold on all MAPS


...

,Precision,Recall,F measure,Accuracy,True Positives,False Positives,False Negatives
H: attack,0.669000,0.661900,0.661200,0.500800,168.100000,89.766700,100.966700
H: normal,0.741400,0.725400,0.730800,0.584000,187.466700,71.033300,81.466700


Piano: StbgTGd2
When averaging each threshold on all MAPS


,Best threshold,Precision,Recall,F measure,Accuracy,True Positives,False Positives,False Negatives
H: attack,0.043000,0.672900,0.684900,0.668500,0.508500,158.400000,86.200000,86.566700
H: normal,0.028000,0.759200,0.749400,0.744800,0.602200,175.733300,62.566700,69.266700


When optimizing the threshold on each song


,Precision,Recall,F measure,Accuracy,True Positives,False Positives,False Negatives
H: attack,0.716000,0.696000,0.702300,0.547900,163.233300,70.833300,81.666700
H: normal,0.766400,0.776400,0.770000,0.633900,186.000000,60.733300,59.000000


### peak fusion

In [13]:
for piano_type in ["AkPnCGdD","ENSTDkCl","AkPnBcht","AkPnBsdf","AkPnStgb","ENSTDkAm","SptkBGAm","StbgTGd2"]:
    compute_scores_database(piano_type, H_normalization = False, fusion_peak=True)

Piano: AkPnCGdD
When averaging each threshold on all MAPS


,Best threshold,Precision,Recall,F measure,Accuracy,True Positives,False Positives,False Negatives
H: attack,0.039000,0.824300,0.799600,0.806300,0.687200,187.000000,47.266700,64.266700
H: normal,0.022000,0.766900,0.652100,0.701300,0.547200,161.200000,52.500000,90.166700


When optimizing the threshold on each song


,Precision,Recall,F measure,Accuracy,True Positives,False Positives,False Negatives
H: attack,0.847100,0.813300,0.828100,0.719000,191.733300,40.766700,59.633300
H: normal,0.806600,0.661200,0.724400,0.575000,163.333300,41.766700,88.033300


Piano: ENSTDkCl
When averaging each threshold on all MAPS


...

,Precision,Recall,F measure,Accuracy,True Positives,False Positives,False Negatives
H: attack,0.818900,0.749800,0.780800,0.650600,192.800000,47.733300,76.266700
H: normal,0.753400,0.656300,0.697700,0.543200,170.666700,62.266700,98.400000


Piano: StbgTGd2
When averaging each threshold on all MAPS


,Best threshold,Precision,Recall,F measure,Accuracy,True Positives,False Positives,False Negatives
H: attack,0.017000,0.799900,0.768500,0.776500,0.645100,181.766700,53.733300,63.100000
H: normal,0.017000,0.789000,0.666400,0.716000,0.566700,159.500000,50.766700,85.566700


When optimizing the threshold on each song


,Precision,Recall,F measure,Accuracy,True Positives,False Positives,False Negatives
H: attack,0.853900,0.755700,0.800000,0.675400,179.933300,33.000000,65.033300
H: normal,0.822200,0.673100,0.738600,0.593800,163.166700,37.666700,81.800000
